In [16]:
from PIL import Image
import numpy as np
from numpy import asarray

# load the image

image = Image.open('cat_posing.jpg')

# convert image to numpy array

data = asarray(image)

print(type(data))

# summarize shape

print(data.shape)



# create Pillow image

image2 = Image.fromarray(data)

print(type(image2))


# summarize image details

print(image2.mode)

print(image2.size)

<class 'numpy.ndarray'>
(512, 512, 3)
<class 'PIL.Image.Image'>
RGB
(512, 512)


In [17]:
print(data)

[[[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 ...

 [[178 136  78]
  [178 136  78]
  [180 138  80]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[168 126  68]
  [172 130  72]
  [185 143  85]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[145 100  43]
  [151 109  51]
  [142  99  44]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]]


In [18]:
# Importing Image and ImageOps module from PIL package
from PIL import Image, ImageOps


# applying grayscale method
im2 = ImageOps.grayscale(image)

im2.show()


In [19]:
brightness = asarray(im2)

In [20]:
total_brightness = sum(sum(brightness))

In [22]:
normalized_brightness= np.divide(brightness,total_brightness)

In [23]:
w = .1 # meters
h = .1 # meters
area_total = w * h

In [128]:
from dolfin import *

In [129]:
mesh = RectangleMesh(Point(0,0), Point(w,h), 512,512)

In [130]:
mesh = RectangleMesh.create([Point(0,0),Point(w,h)],[512,512],CellType.Type.quadrilateral)

In [131]:
V = FunctionSpace(mesh,"CG",1)
D = FunctionSpace(mesh,"DG",0)

In [132]:
br = Function(D)

br.vector().vec().array[:] = brightness.reshape(mesh.num_cells())

In [134]:
with XDMFFile("brightness.xdmf") as xdmf:
    xdmf.write(br)

In [135]:
area = Function(D)
v = TestFunction(D)

area.vector()[:] = assemble(1*v*dx)

In [138]:
area.vector()[:].sum()

0.009999999999999966

In [173]:
loss = br / Constant(total_brightness) - area / Constant(area_total)

In [174]:
loss = project(loss,V)

In [175]:
loss

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 170), FiniteElement('Q', quadrilateral, 1)), 295)

In [152]:
with XDMFFile("loss.xdmf") as xdmf:
    xdmf.write(loss)

In [153]:
gloss = project(grad(loss),VectorFunctionSpace(mesh,"DG",0))

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to assemble system.
*** Reason:  expected a linear form for L.
*** Where:   This error was encountered inside SystemAssembler.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [154]:
with XDMFFile("gloss.xdmf") as xdmf:
    xdmf.write(gloss)

In [155]:
ph =  Function(D)

In [156]:
u = TrialFunction(D)
v = TestFunction(D)

a = inner(grad(u), grad(v))*dx

In [157]:
solve(a == -loss*v*dx,ph)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.


In [158]:
with XDMFFile("phi.xdmf") as xdmf:
    xdmf.write(ph)

In [159]:
ph.vector().min()

-inf

In [160]:
ph.vector().max()

-inf

In [200]:
from dolfin import *

mesh = RectangleMesh.create([Point(0, 0), Point(w, h)],[512,512],CellType.Type.quadrilateral)
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)

In [201]:
# Build function space with Lagrange multiplier
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
R = FiniteElement("Real", mesh.ufl_cell(), 0)
W = FunctionSpace(mesh, P1 * R)

# Define variational problem
(u, c) = TrialFunction(W)
(v, d) = TestFunctions(W)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [206]:
D = FunctionSpace(mesh,"DG",0)
br = Function(D)
br.vector().vec().array[:] = brightness.reshape(mesh.num_cells())

In [209]:
V = FunctionSpace(mesh,"CG",1)
br= project(br,V)

In [222]:
(cr,dr) = Function(W).split()
(ar,er) = Function(W).split()

In [223]:
br.vector().size()

263169

In [224]:
cr.vector().vec().array[0:br.vector().size()] = br.vector()[:]

In [225]:
area = Function(D)
vv = TestFunction(D)

area.vector()[:] = assemble(1*vv*dx)

In [227]:
ar.vector().vec().array[0:area.vector().size()] = area.vector()[:]

In [228]:
loss = cr / Constant(total_brightness) - ar / Constant(area_total)

In [229]:
a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
L = loss*v*dx

# Compute solution
w = Function(W)
solve(a == L, w)
(u, c) = w.split()

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.


In [230]:
with XDMFFile("phi.xdmf") as xdmf:
    xdmf.write(u)